In [1]:
import pandas as pd
import csv
from zipfile import ZipFile

N_DIMS = 300
z = ZipFile("../embeddings/glove6b/glove.6B.zip")
f = z.open(f'glove.6B.{N_DIMS}d.txt')

word_matrix = pd.read_table(
    f, sep=" ", index_col=0, 
    header=None, quoting=csv.QUOTE_NONE
)
word_matrix.head()

,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
0,,,,,,,,,,,,,,,,,,,,,
the,0.046560,0.213180,-0.007436,-0.458540,-0.035639,0.236430,-0.288360,0.215210,-0.134860,-1.6413,...,-0.013064,-0.296860,-0.079913,0.195000,0.031549,0.285060,-0.087461,0.009061,-0.209890,0.053913
",",-0.255390,-0.257230,0.131690,-0.042688,0.218170,-0.022702,-0.178540,0.107560,0.058936,-1.3854,...,0.075968,-0.014359,-0.073794,0.221760,0.146520,0.566860,0.053307,-0.232900,-0.122260,0.354990
.,-0.125590,0.013630,0.103060,-0.101230,0.098128,0.136270,-0.107210,0.236970,0.328700,-1.6785,...,0.060148,-0.156190,-0.119490,0.234450,0.081367,0.246180,-0.152420,-0.342240,-0.022394,0.136840
of,-0.076947,-0.021211,0.212710,-0.722320,-0.139880,-0.122340,-0.175210,0.121370,-0.070866,-1.5721,...,-0.366730,-0.386030,0.302900,0.015747,0.340360,0.478410,0.068617,0.183510,-0.291830,-0.046533
to,-0.257560,-0.057132,-0.671900,-0.380820,-0.364210,-0.082155,-0.010955,-0.082047,0.460560,-1.8477,...,-0.012806,-0.597070,0.317340,-0.252670,0.543840,0.063007,-0.049795,-0.160430,0.046744,-0.070621


In [2]:
from sklearn.metrics.pairwise import cosine_similarity
word_list = ["dog","cat","carbon"]
words = word_matrix.loc[word_list]
sims = pd.DataFrame(cosine_similarity(words))
sims.index, sims.columns = word_list, word_list
sims

,dog,cat,carbon
dog,1.000000,0.681675,0.019671
cat,0.681675,1.000000,0.010330
carbon,0.019671,0.010330,1.000000


In [3]:
# Distance between 2 words
from scipy.spatial.distance import cosine
vec_a = word_matrix.loc["paris"]
vec_b = word_matrix.loc["france"]
1 - cosine(vec_a, vec_b)

0.6580672325437252

In [4]:
# Similar words to x
vec_a = word_matrix.loc["cat"]
sims = 1 - word_matrix.apply(cosine, axis=1, args=(vec_a,))
sims.sort_values(ascending=False).head(6)

0
cat       1.000000
dog       0.681675
cats      0.681584
pet       0.587037
dogs      0.540767
feline    0.489797
dtype: float64

In [5]:
# function for similar words to x
def similar_words(word, word_matrix):
    vec_a = word_matrix.loc[word]
    sims = 1 - word_matrix.apply(cosine, axis=1, args=(vec_a,))
    return sims.sort_values(ascending=False)

similar_words("carbon",word_matrix).head(6)

0
carbon        1.000000
dioxide       0.847373
emissions     0.768768
co2           0.753625
greenhouse    0.701156
gases         0.675826
dtype: float64

In [6]:
diff = word_matrix.loc["paris"] - word_matrix.loc["france"] 
vec_d = word_matrix.loc["berlin"] - diff
sims = 1 - word_matrix.apply(cosine, axis=1, args=(vec_d,))
sims.sort_values(ascending=False).head(6)

0
germany    0.809877
berlin     0.657476
german     0.619871
france     0.616306
austria    0.564680
poland     0.561546
dtype: float64

In [7]:
## Document embeddings

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
docs = [
  "The acclaimed author penned novels based on her life",
  "Nobel prize-winning writer writes autobiographical fiction"
]
vec = CountVectorizer()
dfmat = vec.fit_transform(docs).todense()
dfmat

matrix([[1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1]])

In [9]:
import numpy as np
common_features = set(word_matrix.index) & set(vec.get_feature_names_out())
common_features = list(common_features)
vocab_ids = [vec.vocabulary_[x] for x in common_features]
glove_dfmat = dfmat[:,vocab_ids]
corpus_word_matrix = word_matrix.loc[common_features,]
doc_matrix = glove_dfmat.dot(corpus_word_matrix)
1 - cosine(doc_matrix[0,].A1, doc_matrix[1,].A1)

0.7185793601183077